# Least Squares and Minimization with `scipy.optimize`

The SciPy module [`scipy.optimize`](https://docs.scipy.org/doc/scipy/reference/optimize.html) contains functions for minimization, least squares and root finding techniques (we shall discuss the latter later).

## Linear Least Squares with `scipy.optimize.lsqlinear`

We can use the [`scipy.optimize.lsqlinear`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.lsq_linear.html#scipy.optimize.lsq_linear) function to solve the same problem we were working on in the [Linear Least Squares Minimization](/linear-regression/least-squares) and [Multivariate Linear Least Squares Minimization](/linear-regression/multivariate-least-squares) pages.

Break down using the function with explicit reference to what came before. Also give worked example with Cepheid Variable data

## Non-linear Least Squares with `scipy.optimize.least_squares`

So far we have only considered linear least squares. The reason for this is that minimization for non-linear cases is non-trivial. The function [`scipy.optimize.least_squares`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.least_squares.html#scipy.optimize.least_squares) performs this minimization numerically <!--- Give an outline of this? --->.

Give breakdown on how to use the function. 
- Give example of error squared and chi-squared.
- Give example of 2D non-linear to illustrate the fit (also vary the starting values)

## Minimization?